# Deque overview

Most of the points below comes from the official Python Guide: https://docs.python.org/3/library/collections.html#collections.deque

* Returns a new deque object initialized left-to-right (using append()) with data from iterable => Not affecting the iterable itself. Not in-place.

* Deques stands for double-ended queue, which is a generalisation of stacks (First-in-last-out) vs queue (First-in-first-out)

* Compare to `list`: list is efficient for retrieval of data given an index, and fixed-length operation. But `list` incurs O(n) memory movement costs for `pop(0)` and `insert(0,v)`. `deque` is efficient in insert or pop element from the queue

* When given a `maxlen`, the `deque` is bounded. Once a bounded length deque is full, insertion at one end will kick the element from the other end. 

# Example
* A bounded length deque serves as a sliding window of a fixed length, like below a windows of 3 elements

In [84]:
nums = [i for i in range(10)]
from collections import deque
bounded_deque = deque(maxlen=3)
for num in nums:
    bounded_deque.append(num)
    print(bounded_deque)

deque([0], maxlen=3)
deque([0, 1], maxlen=3)
deque([0, 1, 2], maxlen=3)
deque([1, 2, 3], maxlen=3)
deque([2, 3, 4], maxlen=3)
deque([3, 4, 5], maxlen=3)
deque([4, 5, 6], maxlen=3)
deque([5, 6, 7], maxlen=3)
deque([6, 7, 8], maxlen=3)
deque([7, 8, 9], maxlen=3)


## Problem 1
* Find the top 3 highest scores from a list of scores in an exam


In [70]:
test_scores = [10, 9, 6, 7, 9, 1, 6, 5, 2, 10]

* `deque` doesn't work because it pop out item without sorting internally
* `heapq.nlargest` is the solution

## Problem 2
* Find the last `n` lines of a document (like `tail` function in Unix)

In [71]:
from collections import deque

In [85]:
def tail(file_name, n=10):
    with open(file_name, 'r') as f:
        return deque(f, n)

# Case study of a solution
* [Solution 1](https://leetcode.com/explore/learn/card/fun-with-arrays/521/introduction/3240/discuss/222079/Python-O(N)-10-lines-two-solutions-explained-beats-100)

* [Solution 2](https://leetcode.com/explore/learn/card/fun-with-arrays/521/introduction/3240/discuss/310865/Python:-A-comparison-of-lots-of-approaches!-Sorting-two-pointers-deque-iterator-generator)

### My own comments
* Fastest sort (be it by Python default `sorted()`) could be O(nlogn). But that is on arbitrary list. 
* On an already sorted list, we should aim for something even faster => To make use of available information 

### With `deque` as output 
* Benefit of deque is the O(1) of adding new element (allows us to append elements to the left of answer in O(1) time)
* Converting the `deque` to `list`: `list(answer)`

In [100]:
class Solution(object):
    def sortedSquares(self, A):
        answer = deque()  # Initiate an empty deque object, to store the values in the output
        l, r = 0, len(A) - 1  # two pointers, l, r to the left and right end of the deque
        while l <= r:  # move the two pointers to the center, until they meet each other
            left, right = abs(A[l]), abs(A[r])  # take the absolute values of the two ends
            if left > right:
                 answer.appendleft(left * left)  # take the bigger one to add to the left of the queue
                l += 1  # advance the left end
            else:
                answer.appendleft(right * right)
                r -= 1 # move the right end backward
        return list(answer)

### With list reversal
* Otherwise, it works also with `list` with `reverse` method (which cost O(n) cost of memory movement)

In [ ]:
class Solution(object):
    def sortedSquares(self, A):
        answer = list()  # Initiate an empty deque object, to store the values in the output
        l, r = 0, len(A) - 1  # two pointers, l, r to the left and right end of the deque
        while l <= r:  # move the two pointers to the center, until they meet each other
            left, right = abs(A[l]), abs(A[r])  # take the absolute values of the two ends
            if left > right:
                answer.append(left*left)
                l += 1  # advance the left end
            else:
                answer.append(right*right)
                r -= 1 # move the right end backward
        answer.reverse()  # inplace (no extra memory needed), but it returns nothing
        return answer # so, if we put return answer.reverse() -> this return nothing

### Without `deque` or `list.reversal`
* Make use of the pointers to arrive at the right location of the result array

In [106]:
class Solution(object):
    def sortedSquares(self, A):
        answer = [None] * len(A) # Initiate an empty deque object, to store the values in the output
        l, r = 0, len(A) - 1  # two pointers, l, r to the left and right end of the deque
        while l <= r:  # move the two pointers to the center, until they meet each other
            left, right = abs(A[l]), abs(A[r])  # take the absolute values of the two ends
            if left > right:
                answer[r-l] = left*left
                l += 1  # advance the left end
            else:
                answer[r-l] = right*right
                r -= 1 # move the right end backward

        return answer # so, if we put return answer.reverse() -> this return nothing

### Space complexity of builtin solution
* If we want to minimize the space => **in-place** modifying the input array. No need to create extra space for the saving 
* Different behaviour of `sorted(Array)` vs `Array.sort()`
    - `sorted(Array)` is not in-place. This returns a new array
    - `Array.sort()` return nothing because it is in-place. The content of the array is overwritten. 

### With `deque` on input
* The elegance will reduce the risk of bug, lead to readable and maintainable code

In [122]:
class Solution(object):
    def sortedSquares(self, A):
        number_deque = deque(A)  # A is left untouched after this point. But number_deque is popped every time
        reverse_sorted_squares = []  # Later to be appended
        while number_deque:  # If the queue is not empty
            left_squares = number_deque[0] ** 2  # power of 2
            right_squares = number_deque[-1] ** 2  
            if left_squares > right_squares:
                reverse_sorted_squares.append(right_squares)  # Adding the smaller one
                number_deque.popleft()
            else:
                reverse_sorted_squares.append(left_squares)  
                number_deque.pop()
        reverse_sorted_squares.reverse()
        print(A)
        return reverse_sorted_squares

In [123]:
nums = [-4,-1,0,3,10]
# Expected utputs = [0, 1, 9, 16, 100]
sol = Solution()
sol.sortedSquares(nums)

[-4, -1, 0, 3, 10]


[0, 0, 1, 9, 16]